In [1]:
from metrics import get_metrics
from metrics import *
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/data1.csv')

In [3]:
df = filter_every_n_min(df,5,timestamp_col)
#Add Exchange-contract column
df[exchange_contract_col] = df[exchange_col]+df[contract_col]
#Clean columns
df[basis_col] = df[basis_col].replace('[\$,\(\)]', '', regex=True).astype(float) 

#Get the percent change of all the Exchange Contract pairs
percent_change_df = get_percent_change(df,timestamp_col,exchange_contract_col,basis_col)
btc_price_percent_change = get_percent_change(df,timestamp_col,exchange_contract_col,current_btc_price_col)['BinanceBTCUSDT_230331']

In [4]:
percent_change_df.stack().abs().describe()

count    9184.000000
mean        0.545714
std         8.134101
min         0.000000
25%         0.025798
50%         0.067512
75%         0.206495
max       549.239398
dtype: float64

In [5]:
percent_change_thresholds = np.arange(0.01,0.2,0.01)

In [6]:
#Loop through different thresolds and get the metrics
df_list = []
for percent_change_threshold in percent_change_thresholds:
    metrics_df = get_metrics(
        percent_change_df = percent_change_df,
        btc_price_percent_change = btc_price_percent_change,
        actual_profit_thresh = percent_change_threshold,
        correlation_thresh = 0)
    df_list.append(metrics_df)

In [7]:
output_df = pd.concat(df_list,keys=percent_change_thresholds)
output_df

exchange-contract   BinanceBTCUSDT_230331  BinanceBTCUSD_230331  \
0.01 correlation                 0.230845              0.237799   
     hit_rate                    0.655052              0.588850   
     actual_profit               0.103115              0.062563   
     profit_ratio                1.306864              0.988274   
     profit_factor               1.884615              1.753086   
...                                   ...                   ...   
0.19 correlation                 0.230845              0.237799   
     hit_rate                    0.655052              0.588850   
     actual_profit               0.000000              0.000000   
     profit_ratio                     NaN                   NaN   
     profit_factor                    NaN                   NaN   

exchange-contract   BinanceBTCUSD_230630  BitMEXXBTF23  BitMEXXBTG23  \
0.01 correlation                0.088731     -0.032293     -0.502182   
     hit_rate                   0.498258      0.351916      0.480836   
     actual_profit              0.053414     -0.070084      0.068103   
     profit_ratio               1.396613      1.057786      1.730657   
     profit_factor              1.175258      0.565476      0.949275   
...                                  ...           ...           ...   
0.19 correlation                0.088731     -0.032293     -0.502182   
     hit_rate                   0.498258      0.351916      0.480836   
     actual_profit              0.000000      0.019608      0.075688   
     profit_ratio                    NaN      0.524210      1.763072   
     profit_factor                   NaN      2.555556      1.548387   

exchange-contract   BitMEXXBTH23  BitMEXXBTM23  BitMEXXBTU23  \
0.01 correlation       -0.128857     -0.016106     -0.068997   
     hit_rate           0.557491      0.703833      0.780488   
     actual_profit      0.031080      0.116979      0.147189   
     profit_ratio       0.980180      1.006137      0.882104   
     profit_factor      1.285714      2.487179      3.678571   
...                          ...           ...           ...   
0.19 correlation       -0.128857     -0.016106     -0.068997   
     hit_rate           0.557491      0.703833      0.780488   
     actual_profit      0.012376      0.061840      0.022139   
     profit_ratio       1.266921      0.882268      1.613794   
     profit_factor      1.060606      2.903226      0.920000   

exchange-contract   BitMEXXBTUSDTH23  BitMEXXBTUSDTM23  ...  \
0.01 correlation           -0.006637         -0.045844  ...   
     hit_rate               0.209059          0.445993  ...   
     actual_profit         -0.129759          0.026269  ...   
     profit_ratio           1.535506          1.328403  ...   
     profit_factor          0.235294          0.912698  ...   
...                              ...               ...  ...   
0.19 correlation           -0.006637         -0.045844  ...   
     hit_rate               0.209059          0.445993  ...   
     actual_profit         -0.008488          0.065816  ...   
     profit_ratio           1.026525          0.537459  ...   
     profit_factor          0.864865          7.200000  ...   

exchange-contract   KrakenFI_XBTUSD_230331  KrakenFI_XBTUSD_230630  \
0.01 correlation                 -0.031191               -0.216804   
     hit_rate                     0.759582                0.871080   
     actual_profit                0.113404                0.190399   
     profit_ratio                 0.706982                0.619427   
     profit_factor                3.338710                8.629630   
...                                    ...                     ...   
0.19 correlation                 -0.031191               -0.216804   
     hit_rate                     0.759582                0.871080   
     actual_profit                0.044082                0.061886   
     profit_ratio                 0.988883                3.011044   
     profit_factor                

In [8]:
#Rename columns for report
rename_col = {
    'level_0':"Percent Change Threshold",
    'level_1':"Metrics"
}
report_df = output_df.reset_index()

report_df.rename(columns=rename_col,inplace=True)
duplicated_mask = report_df['Percent Change Threshold'].duplicated()
report_df['Percent Change Threshold'][duplicated_mask] = ''
report_df.to_csv('report.csv',index=False)
report_df

C:\Users\jorda\AppData\Local\Temp\ipykernel_15792\2783187530.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_df['Percent Change Threshold'][duplicated_mask] = ''


exchange-contract,Percent Change Threshold,Metrics,BinanceBTCUSDT_230331,BinanceBTCUSD_230331,BinanceBTCUSD_230630,BitMEXXBTF23,BitMEXXBTG23,BitMEXXBTH23,BitMEXXBTM23,BitMEXXBTU23,...,KrakenFI_XBTUSD_230331,KrakenFI_XBTUSD_230630,OKXBTC-USD-230127,OKXBTC-USD-230203,OKXBTC-USD-230331,OKXBTC-USD-230630,OKXBTC-USDT-230127,OKXBTC-USDT-230203,OKXBTC-USDT-230331,OKXBTC-USDT-230630
0,0.01,correlation,0.230845,0.237799,0.088731,-0.032293,-0.502182,-0.128857,-0.016106,-0.068997,...,-0.031191,-0.216804,-0.079070,0.424989,0.024486,0.249954,0.439611,0.410028,-0.056100,0.210009
1,,hit_rate,0.655052,0.588850,0.498258,0.351916,0.480836,0.557491,0.703833,0.780488,...,0.759582,0.871080,0.452962,0.585366,0.682927,0.658537,0.540070,0.620209,0.397213,0.599303
2,,actual_profit,0.103115,0.062563,0.053414,-0.070084,0.068103,0.031080,0.116979,0.147189,...,0.113404,0.190399,-0.030649,0.094529,0.115216,0.109074,0.043894,0.083578,-0.084183,0.080990
3,,profit_ratio,1.306864,0.988274,1.396613,1.057786,1.730657,0.980180,1.006137,0.882104,...,0.706982,0.619427,0.966591,1.391947,1.175169,1.277834,1.145358,1.160343,0.863444,1.312399
4,,profit_factor,1.884615,1.753086,1.175258,0.565476,0.949275,1.285714,2.487179,3.678571,...,3.338710,8.629630,0.832258,1.477477,2.291667,2.300000,1.193798,1.609524,0.569536,1.590361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0.19,correlation,0.230845,0.237799,0.088731,-0.032293,-0.502182,-0.128857,-0.016106,-0.068997,...,-0.031191,-0.216804,-0.079070,0.424989,0.024486,0.249954,0.439611,0.410028,-0.056100,0.210009
91,,hit_rate,0.655052,0.588850,0.498258,0.351916,0.480836,0.557491,0.703833,0.780488,...,0.759582,0.871080,0.452962,0.585366,0.682927,0.658537,0.540070,0.620209,0.397213,0.599303
92,,actual_profit,0.000000,0.000000,0.000000,0.019608,0.075688,0.012376,0.061840,0.022139,...,0.044082,0.061886,-0.024417,0.058415,0.016495,0.013794,0.035502,0.060065,-0.006676,0.013794
93,,profit_ratio,NaN,NaN,NaN,0.524210,1.763072,1.266921,0.882268,1.613794,...,0.988883,3.011044,0.989430,1.342727,5.675785,9.656499,1.151078,1.298902,0.571160,9.656499


In [14]:
#Only get the profit ratio
profit_ratio_df = output_df.reset_index()
profit_ratio_df = profit_ratio_df[profit_ratio_df['level_1'] == 'profit_factor'].set_index('level_0').drop('level_1',axis=1)
profit_ratio_df

exchange-contract,BinanceBTCUSDT_230331,BinanceBTCUSD_230331,BinanceBTCUSD_230630,BitMEXXBTF23,BitMEXXBTG23,BitMEXXBTH23,BitMEXXBTM23,BitMEXXBTU23,BitMEXXBTUSDTH23,BitMEXXBTUSDTM23,...,KrakenFI_XBTUSD_230331,KrakenFI_XBTUSD_230630,OKXBTC-USD-230127,OKXBTC-USD-230203,OKXBTC-USD-230331,OKXBTC-USD-230630,OKXBTC-USDT-230127,OKXBTC-USDT-230203,OKXBTC-USDT-230331,OKXBTC-USDT-230630
level_0,,,,,,,,,,,,,,,,,,,,,
0.01,1.884615,1.753086,1.175258,0.565476,0.949275,1.285714,2.487179,3.678571,0.235294,0.912698,...,3.338710,8.629630,0.832258,1.477477,2.291667,2.300000,1.193798,1.609524,0.569536,1.590361
0.02,2.411765,1.919355,1.164179,0.606667,0.954887,1.260870,2.700000,4.020833,0.245810,1.049505,...,3.500000,11.157895,0.836601,1.519231,2.519231,2.484848,1.186047,1.670000,0.579832,1.654545
0.03,2.424242,1.977273,1.125000,0.659091,1.000000,1.326923,2.800000,3.595745,0.274510,1.417910,...,3.363636,11.562500,0.834437,1.581633,2.350000,3.133333,1.192000,1.715789,0.604167,1.500000
0.04,2.318182,2.096774,1.076923,0.704348,0.973214,1.313131,2.781250,3.250000,0.310078,1.872340,...,3.307692,10.333333,0.827815,1.608696,2.000000,1.888889,1.206612,1.733333,0.479452,1.416667
0.05,2.187500,2.166667,1.545455,0.782178,0.990291,1.355556,2.774194,3.069767,0.351351,2.484848,...,3.280000,9.500000,0.826667,1.686747,1.916667,2.333333,1.201681,1.781609,0.460000,1.210526
0.06,1.666667,2.333333,0.875000,0.883721,0.958763,1.392857,2.750000,2.690476,0.383838,2.703704,...,3.104167,9.500000,0.829932,1.701299,1.500000,2.500000,1.232759,1.744186,0.483871,1.000000
0.07,1.000000,1.625000,1.000000,0.972222,1.108434,1.389610,2.724138,2.317073,0.415730,4.533333,...,3.162791,11.666667,0.828767,1.774648,1.461538,1.000000,1.232143,1.768293,0.434783,1.000000
0.08,1.250000,2.000000,0.000000,1.096774,1.219178,1.380282,2.684211,2.128205,0.467532,6.090909,...,3.175000,9.666667,0.834483,1.820896,1.750000,1.000000,1.254545,1.753086,0.375000,0.833333
0.09,1.000000,2.666667,0.000000,1.245283,1.296875,1.363636,2.740741,2.000000,0.500000,6.666667,...,2.800000,11.428571,0.839161,1.796875,1.250000,1.000000,1.261682,1.807692,0.461538,1.000000


In [19]:
#Loop through the differnt profit ratios
n = 5
results = []
for percent_thresh,row in profit_ratio_df.iterrows():
    #Get the top n profit ratio
    top_n_index = row.argsort()[-n:][::-1]
    top_n = row[top_n_index]

    #Filter the initial percent_change_df by the top_n exchange-contract pair
    votes_df = percent_change_df[top_n.index]
    #Convert to 1 or -1
    votes_df = votes_df/votes_df.abs()

    #The final vote is the sum of the votes
    final_vote = votes_df.sum(axis=1)

    #Convert to 1 or -1
    final_vote = final_vote/final_vote.abs()
    
    #Calculate for metrics
    hit_rate = calc_hit_rate(final_vote,btc_price_percent_change)
    trades_df = calc_actual_profit(final_vote,btc_price_percent_change)
    actual_profit = trades_df.sum()
    profit_ratio = calc_profit_ratio(trades_df)
    profit_factor = calc_profit_factor(trades_df)
    results.append([hit_rate,actual_profit,profit_ratio,profit_factor])



In [11]:
results_df = pd.DataFrame(results,columns=['Hit rate','Actual Profit','Profit Ratio', 'Profit Factor'],index=profit_ratio_df.index.tolist())
results_df.index.name = 'Thresholds'


In [13]:
results_df

,Hit rate,Actual Profit,Profit Ratio,Profit Factor
Thresholds,,,,
0.01,0.160279,-0.162003,1.431153,0.190871
0.02,0.198606,-0.123583,1.586779,0.247826
0.03,0.188153,-0.128535,1.624584,0.231760
0.04,0.149826,-0.165557,1.492924,0.176230
0.05,0.156794,-0.164177,1.435804,0.185950
0.06,0.149826,-0.167959,1.454811,0.176230
0.07,0.170732,-0.155581,1.417207,0.205882
0.08,0.149826,-0.167281,1.465531,0.176230
0.09,0.156794,-0.165756,1.411867,0.185950


In [ ]:
#TODO Handle infinites
# 